# 🔥 1. What is Gradient Boosting?

**Gradient Boosting is an ensemble(jorna) technique where:**
  * Models (typically decision trees) are added one at a time.

  * Each new tree tries to correct the errors made by the previous trees.

  * It minimizes a loss function using gradient descent.

**🧠 Think of it as:**
"Make a prediction → See where you're wrong → Train a new tree to fix those errors → Repeat."



### 🛠 Gradient Boosting Flow:
  * Start with an initial model (say, mean of target).

  * Calculate the residuals (errors).

  * Train a tree on the residuals.

  * Add this tree to the model prediction.

  * Repeat the process multiple times.

## Gradient Boosting using GradientBoostingClassifier
**We'll use the Breast Cancer dataset from sklearn:**